# 🎓 BDH Training for Brain Explorer

This notebook trains a Baby Dragon Hatchling (BDH) model on the pathfinding task to achieve **~5% sparsity**.

**Expected Time**: 2-4 hours on T4 GPU

**Steps**:
1. Clone krychu/bdh repository
2. Install dependencies
3. Train BDH model
4. Save checkpoint
5. Download `bdh_trained.pth`

## Step 1: Clone Repository

In [ ]:
!git clone https://github.com/krychu/bdh.git
%cd bdh

## Step 2: Install Dependencies

In [ ]:
!pip install torch numpy matplotlib networkx pillow -q

## Step 3: Import Libraries

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from bdh import BDH, BDHParameters
from boardpath import BoardPathDataset, BoardPathConfig
import time

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## Step 4: Configure Model

In [ ]:
# Model configuration matching Brain Explorer
params = BDHParameters(
    V=5,          # Vocabulary: 0=empty, 1=wall, 2=start, 3=end, 4=path
    T=100,        # Sequence length (10x10 board flattened)
    H=4,          # Number of heads
    N=2048,       # Number of neurons (sparse layer)
    D=64,         # Latent dimension
    L=12,         # Number of layers
    dropout=0.1,
    use_rope=True,
    use_abs_pos=False
)

print(f"Model Configuration:")
print(f"  Neurons: {params.N}")
print(f"  Layers: {params.L}")
print(f"  Heads: {params.H}")
print(f"  Latent Dim: {params.D}")

## Step 5: Create Dataset

In [ ]:
# Dataset configuration
config = BoardPathConfig(
    board_size=10,
    num_examples=10000,  # 10k training examples
    vocab_size=5,
    seq_len=100
)

dataset = BoardPathDataset(config)
loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=32, 
    shuffle=True,
    num_workers=2
)

print(f"Dataset: {len(dataset)} examples")
print(f"Batches per epoch: {len(loader)}")

## Step 6: Initialize Model & Optimizer

In [ ]:
model = BDH(params)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## Step 7: Training Loop

In [ ]:
num_epochs = 50  # Adjust based on convergence
losses = []
best_loss = float('inf')

print("🚀 Starting Training...")
print("=" * 60)

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    for batch_idx, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        logits, _ = model(x)
        
        # Compute loss
        loss = criterion(logits.view(-1, params.V), y.view(-1))
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
        
        # Log progress
        if batch_idx % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{len(loader)} | Loss: {loss.item():.4f}")
    
    # Epoch summary
    avg_loss = epoch_loss / len(loader)
    losses.append(avg_loss)
    
    elapsed = time.time() - start_time
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1} Complete | Avg Loss: {avg_loss:.4f} | Time: {elapsed/60:.1f}m")
    print(f"{'='*60}\n")
    
    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'bdh_best.pth')
        print(f"✅ New best model saved! Loss: {best_loss:.4f}\n")
    
    # Early stopping if loss is very low
    if avg_loss < 0.05:
        print("🎉 Training converged! Loss < 0.05")
        break

print("\n" + "="*60)
print("✅ Training Complete!")
print(f"Total time: {(time.time() - start_time)/60:.1f} minutes")
print(f"Best loss: {best_loss:.4f}")
print("="*60)

## Step 8: Save Final Checkpoint

In [ ]:
# Save final model
torch.save(model.state_dict(), 'bdh_trained.pth')
print("✅ Final model saved: bdh_trained.pth")

# Also save the best model with a clear name
!cp bdh_best.pth bdh_trained.pth
print("✅ Best model copied to: bdh_trained.pth")

## Step 9: Verify Sparsity

In [ ]:
# Test sparsity on a sample
model.eval()
with torch.no_grad():
    # Get a test sample
    x_test, _ = dataset[0]
    x_test = x_test.unsqueeze(0).to(device)
    
    # Forward pass with state tracking
    logits, output_frames, x_frames, y_frames, attn_frames, logits_frames = model(x_test, capture_frames=True)
    
    # Compute sparsity
    if y_frames:
        # Average sparsity across all layers
        sparsities = []
        for layer_activations in y_frames:
            active = (layer_activations > 0).float().mean().item()
            sparsities.append(active * 100)
        
        avg_sparsity = np.mean(sparsities)
        print(f"\n🎯 Sparsity Analysis:")
        print(f"  Average sparsity: {avg_sparsity:.2f}%")
        print(f"  Target: ~5%")
        print(f"  Status: {'✅ GOOD' if avg_sparsity < 10 else '⚠️ Needs more training'}")
        
        print(f"\n  Per-layer sparsity:")
        for i, s in enumerate(sparsities):
            print(f"    Layer {i+1}: {s:.2f}%")

## Step 10: Plot Training Loss

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(losses, linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('BDH Training Loss', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('training_loss.png', dpi=150)
plt.show()

print(f"Final loss: {losses[-1]:.4f}")
print(f"Best loss: {min(losses):.4f}")

## ✅ Training Complete!

### Next Steps:

1. **Download the checkpoint**:
   - Look in the **Output** section (right sidebar)
   - Find `bdh_trained.pth`
   - Click the three dots → Download

2. **Deploy to Brain Explorer**:
   ```bash
   # In your local project
   mkdir -p checkpoints
   mv ~/Downloads/bdh_trained.pth checkpoints/
   
   # Restart backend
   cd backend/api
   python app.py
   ```

3. **Verify**:
   - Backend should show: `🎓 TRAINED MODEL MODE`
   - Frontend banner should be green: "Trained Model Active"
   - Sparsity should be ~5% instead of ~25%

🎉 **Congratulations! You now have a production-grade trained BDH model!**